# CONFIG

#### Imports

In [1]:
import os
import math
import random
import numpy as np
import pandas as pd
import requests
import csv
import time
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import matplotlib.image as mpimg #another version of opencv display image. I think it's just rendering things that have a height, width, and color dimensionality to them

from tensorflow.keras import layers
from tensorflow.keras import Model

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator #using because this is a cnn and we want it to handle images in a fast manner
from tensorflow.keras.preprocessing.image import img_to_array, load_img

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Special
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

#### Pathing

In [2]:
url = 'https://api.pushshift.io/reddit/search/submission'

#### Toggles

In [4]:
BATCH_SIZE = 20
IMG_SHAPE = 150 #same shape for height and width
EPOCHS = 15 #how many iterations our model will run
RANDOM_STATE = 54321
np.random.seed(RANDOM_STATE)

In [70]:
n = 5
y_df = pd.DataFrame()
while len(y_df) < 3:
    y = str(n**2)
    n -=1
    y_df = y_df.append(pd.DataFrame({"A": [y, y]}))
    print(y_df, len(y_df))

    A
0  25
1  25 2
    A
0  25
1  25
0  16
1  16 4


In [63]:
base_url = 'https://api.pushshift.io/reddit/search/submission/?subreddit='
subreddit = 'physics'
current_time =1602126873
df_add_to = pd.DataFrame()
while len(df_add_to) < 4:
    res = requests.get(
                # requests.get takes base_url and params
                base_url,
                # parameters for get request
                params = {
                    # specify subreddit
                    'subreddit' : subreddit,
                    # specify number of posts to pull
                    'size' : 10,
                    # restrict based on the default language of the subreddit
                    'lang' : True,
                    # pull everything from current time backward
                    'before' : current_time }
            )

    # take data from most recent request, store as df
    df = pd.DataFrame(res.json()['data'])
    # pull specific columns from dataframe for analysis
    df = df.loc[:, ['subreddit', 'selftext', 'title', 'created_utc']]
    #only keep rows that do not have empty values for 'selftext' or 'title'
    df_drop = df.loc[df['selftext'].str.split().str.len() > 4]
    df_drop = df_drop.loc[df_drop['title'].str.split().str.len() > 4]
    #make sure we didn't accidentally duplicate any rows
    df_drop = df_drop.drop_duplicates()

    # append to empty dataframe 
    df_add_to = df_add_to.append(df_drop)
    
    #TEST: calculate time difference
    time_diff = current_time - df_add_to['created_utc'].min()

    #reset time 
    current_time -= time_diff
    
    print('time_diff', time_diff)
    print('current_time', current_time)

time_diff 15877
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time 1602110996
time_diff 0
current_time

KeyboardInterrupt: 

In [56]:
type(current_time)

int

In [52]:
df_add_to

,subreddit,selftext,title,created_utc
2,Physics,"Hey everyone, I need a design for a catapult t...",What is a basic catapult that I can design for...,1602117592
7,Physics,"If I were to accelerate a charge, over a perio...",Extra work due to accelerating a charge,1602110996


In [48]:
len(df_add_to)

2

In [31]:
def get_posts(subreddit, n_iter, epoch_right_now): # subreddit name and number of times function should run
    # store base url variable
    base_url = 'https://api.pushshift.io/reddit/search/submission/?subreddit='
    # instantiate empty list    
    df_list = []
    #instantiate empty dataframe
    df_add_to = pd.DataFrame()
    # save current epoch, used to iterate in reverse through time
    current_time = epoch_right_now
    # set up while loop
    while len(df_add_to) < 6:
        # instantiate get request
        res = requests.get(
            # requests.get takes base_url and params
            base_url,
            # parameters for get request
            params = {
                # specify subreddit
                'subreddit' : subreddit,
                # specify number of posts to pull
                'size' : 2,
                # ???
                'lang' : True,
                # pull everything from current time backward
                'before' : current_time }
        )
        # take data from most recent request, store as df
        df = pd.DataFrame(res.json()['data'])
        # pull specific columns from dataframe for analysis
        df = df.loc[:, ['subreddit', 'selftext', 'title', 'created_utc']]
        #only keep rows that do not have empty values for 'selftext' or 'title'
        df_drop = df.loc[df['selftext'].str.split().str.len() > 4]
        df_drop = df_drop.loc[df_drop['title'].str.split().str.len() > 4]
        #make sure we didn't accidentally duplicate any rows
        df_drop = df_drop.drop_duplicates()
        # append to empty dataframe 
        df_add_to = df_add_to.append(df_drop)
        # add wait time
        time.sleep(5)
        # set current time counter back to last epoch in recently grabbed df
        current_time = df_drop['created_utc'].min()
    # return one dataframe for all requests
    #return pd.concat(df_list, axis=0)
    return df_add_to
# Adapated from Tim Book's Lesson Example

In [32]:
df = get_posts('physics', 4, 1602126873)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df.info()

In [24]:
df_test = pd.DataFrame()
df_test = df_test.append(pd.DataFrame({"A": range(3)}))
df_test

,A
0,0
1,1
2,2


In [35]:
df_test = pd.DataFrame()
while len(df_test) < 13:
    df_drop = pd.DataFrame({"A": range(3)})
    df_test = df_test.append(df_drop)

In [36]:
df_test

,A
0,0
1,1
2,2
0,0
1,1
2,2
0,0
1,1
2,2
0,0


In [37]:
len(df_test)

15